# ID3

ID3 algorithm, stands for **Iterative Dichotomiser 3**, is a **classification algorithm** that follows a **greedy approach** of **building a decision tree** by select best attribute that yields **maximum Information Gain (IG)** or **minimum Entropy (H)**.

## Decision Tree

A decision tree is a tree where each

- Node - a feature (attribute)
- Branch - a decision (rule)
- Leaf - an outcome (categorical or continuous)

## Entropy

Entropy $H(S)$ is a measure of the amount of uncertainty in the (data) set $S$ (i.e. entropy characterize the (date) set $S$).

> 熵是对(数据)集中的不确定性量的度量.

$$
H(S) = \sum_{x \in X}-p(x)log_{2}p(x)
$$

Where,

- $S$ - The current dataset for which entropy is being calculated -- 需要被计算熵的数据集
- $X$ - The set of classes in $S$ -- 数据集中的分类
- $p(x)$ - The proportion of the number of elements in class $x$ to the number of elements in set $S$ -- $x$ 分类中元素的数据与数据集中的总元素梳理的比

When $H(S) = 0$, the set $S$ is perfectly classified (i.e. all elements in $S$ are of the same class).

## Information gain

Information gain $IG(A)$ is the measure of the difference in entropy from before to after the set $S$ is split on an attribute $A$. In other words, how much uncertainty in $S$ was reduced after splitting set $S$ on attribute $A$.

> 信息增益是在属性 $A$ 上分割集合 $S$ 之前和之后的熵差的度量. 换句话说, 属性 $A$ 上分割集合 $S$ 后, $S$ 中的不确定性降低了多少.

$$
\begin{align*}
  IG(S, A) &= H(S) - \sum_{t \in T} p(t) H(t) \\
           &= H(S) - H(S|A)
\end{align*}
$$

Where,

- $H(S)$ - Entropy of set $S$
- $T$ - The subsets created from splitting set $S$ by attribute $A$ such that $S = \displaystyle \bigcup_{t \in T}t$
- $p(t)$ - The proportion of the number of elements in $t$ to the number of elements in set $S$
- $H(t)$ - Entropy of subset $T$

## Steps in ID3 Algorithm

1. Calculate entropy for dataset.
2. For each attribute/feature.
   1. Calculate entropy for all its categorical values.
   2. Calculate information gain for the feature.
3. Find the feature with maximum information gain.
4. Repeat it until we get the desired tree.

## Example

这里我们使用一个天气数据集来解释如何使用 ID3 算法做决策树模式.

建立决策树的目的是根据天气的情况来判断是否可以去打网球, 也就是建立一个根据天气情况来预测是否可以去打网球的模型.

创建数据集 $S$


In [3]:
import pandas as pd
import numpy as np


In [4]:
S = pd.DataFrame({
    "Day": list(range(1, 15)),
    "Outlook": ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain',
                'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain',
                'Sunny', 'Overcast', 'Overcast', 'Rain'],
    "Temperature": ['Hot', 'Hot', 'Hot', 'Mild', 'Cool',
                    'Cool', 'Cool', 'Mild', 'Cool', 'Mild',
                    'Mild', 'Mild', 'Hot', 'Mild'],
    "Humidity": ['High', 'High', 'High', 'High', 'Normal',
                 'Normal', 'Normal', 'High', 'Normal', 'Normal',
                 'Normal', 'High', 'Normal', 'High'],
    "Wind": ['Weak', 'Strong', 'Weak', 'Weak', 'Weak',
             'Strong', 'Strong', 'Weak', 'Weak', 'Weak',
             'Strong', 'Weak', 'Strong', 'Strong'],
    "Play Tennis": ['No', 'No', 'Yes', 'Yes', 'Yes',
                    'No', 'Yes', 'No', 'Yes', 'Yes',
                    'Yes', 'Yes', 'Yes', 'No']
})
S


,Day,Outlook,Temperature,Humidity,Wind,Play Tennis
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes
5,6,Rain,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rain,Mild,Normal,Weak,Yes


### Step 1: 计算 $H(S)$

计算整个数据集的熵, 使用的是标签列, 也就是预测结果, 这里指的是 'Play Tennis'.


In [5]:
# count by 'Play Tennis'
hs = S.groupby('Play Tennis')[['Day']]\
    .count()\
    .rename(columns={'Day': 'Count'})
# calculate p(x)
hs['p(x)'] = hs['Count'] / hs.sum()['Count']
# calculate H(x)
hs['H(x)'] = -1 * hs['p(x)'] * np.log2(hs['p(x)'])
hs


,Count,p(x),H(x)
Play Tennis,,,
No,5,0.357143,0.530510
Yes,9,0.642857,0.409776


In [6]:
# calculate H(S)
print(f'H(S) = {hs.sum()["H(x)"]}')


H(S) = 0.9402859586706311


将数据带入熵计算公式:

$$
\begin{align*}
  H(S) &= -p(yes) * log_2(p(yes)) - p(no) * log_2(p(no)) \\
       &= -(9/14) * log_2((9/14)) - (5/14) * log_2((5/14)) \\
       &= -(0.41) - (-0.53) \\
       &= 0.94
\end{align*}
$$


### Step 2: 计算 "Outlook" attribute/feature 的信息增益 $IG(S, Outlook)$

#### Step 2.1: 计算 'Outlook' attribute 的每个类别的熵 H(t)


In [7]:
# count by 'Outlook' and 'Play Tennis'
ht = S.groupby(['Outlook', 'Play Tennis'])[['Day']]\
    .count()\
    .rename(columns={'Day': 'Count'})
# calculate p(t)
ht['p(t)'] = ht.groupby(level=0)\
    .apply(lambda x: x / x.sum())['Count']
# calculate H(t)
ht['H(t)'] = -1 * ht['p(t)'] * np.log2(ht['p(t)'])
# calculate I(t)
ht = ht.groupby(level=0).sum()
ht['p(t)'] = ht['Count'] / ht.sum()['Count']
ht['I(t)'] = ht['p(t)'] * ht['H(t)']
ht


,Count,p(t),H(t),I(t)
Outlook,,,,
Overcast,4,0.285714,0.000000,0.000000
Rain,5,0.357143,0.970951,0.346768
Sunny,5,0.357143,0.970951,0.346768


> Categorical values - sunny, overcast and rain
>
> $H(Outlook=sunny) = -(2/5)*log_2(2/5) - (3/5)*log(3/5) = 0.971$
>
> $H(Outlook=rain) = -(3/5)*log_2(3/5) - (2/5)*log(2/5) = 0.971$
>
> $H(Outlook=overcast) = -(4/4)*log_2(4/4) - (4/4)*log(4/4) = 0$


In [8]:
print(f'I(Outlook) = {ht.sum()["I(t)"]}')


I(Outlook) = 0.6935361388961918


计算 'Outlook' 的平均信息熵, $I(Outlook)$

$$
\begin{align*}
  I(Outlook) &= p(sunny) * H(Outlook=sunny) + p(rain) * H(Outlook=rain) + p(overcast) * H(Outlook=overcast) \\
             &= (5/14)*0.971 + (5/14)*0.971 + (4/14) * 0 \\
             &= 0.693
\end{align*}
$$


#### Step 2.2: 计算 "Outlook" feature 的信息增益 $IG(S, Outlook)$

$$
\begin{align*}
  IG(S, Outlook) &= H(S) - I(Outlook) \\
                 &= 0.94 - 0.693 \\
                 &= 0.247
\end{align*}
$$

接着我们需要使用同样的方法计算, $IG(S, Temperature), IG(S, Humidity), IG(S, Wind)$

为了简便, 我们创建一个函数来计算 $IG$


In [9]:
def calculateEntropy(df_, class_, count_):
    '''计算数据集的熵
    @df_::DataFrame: 数据集
    @class_::String: 分类结果字段名称
    @count_::String: 用于计数的字段名称
    '''
    # count
    hs = df_.groupby(class_)[[count_]]\
        .count()\
        .rename(columns={'Day': 'Count'})
    # calculate p(x)
    hs['p(x)'] = hs['Count'] / hs.sum()['Count']
    # calculate H(x)
    hs['H(x)'] = -1 * hs['p(x)'] * np.log2(hs['p(x)'])
    return {
        'df': hs,
        'value': hs.sum()['H(x)']
    }


def averageInformationEntropy(feature_, df_, class_, count_):
    '''计算平均信息熵
    @feature_::String: 需要计算平均信息熵的特征字段名称
    @df_::DataFrame: 数据集
    @class_::String: 分类结果字段名称
    @count_::String: 用于计数的字段名称
    '''
    # count
    ht = df_.groupby([feature_, class_])[[count_]]\
        .count()\
        .rename(columns={count_: 'Count'})
    # calculate p(t)
    ht['p(t)'] = ht.groupby(level=0)\
        .apply(lambda x: x / x.sum())['Count']
    # calculate H(t)
    ht['H(t)'] = -1 * ht['p(t)'] * np.log2(ht['p(t)'])
    # calculate I(t)
    ht = ht.groupby(level=0).sum()
    ht['p(t)'] = ht['Count'] / ht.sum()['Count']
    ht['I(t)'] = ht['p(t)'] * ht['H(t)']
    return {
        'df': ht,
        'value': ht.sum()['I(t)']
    }


### Step 3: 找出信息增加最大的 feature.

信息增益最大的是 Outlook = 0.94 - 0.693 = 0.247


In [15]:
entropy = calculateEntropy(df_=S, class_='Play Tennis', count_='Day')
entropy['df']


,Count,p(x),H(x)
Play Tennis,,,
No,5,0.357143,0.530510
Yes,9,0.642857,0.409776


In [21]:
print(f'S 集合的信息熵是: {entropy["value"]}')


S 集合的信息熵是: 0.9402859586706311


In [18]:
# calculate Information gain for "Outlook"
outlook_average_entropy = averageInformationEntropy(
    df_=S, class_='Play Tennis', count_='Day', feature_='Outlook')
outlook_average_entropy['df']


,Count,p(t),H(t),I(t)
Outlook,,,,
Overcast,4,0.285714,0.000000,0.000000
Rain,5,0.357143,0.970951,0.346768
Sunny,5,0.357143,0.970951,0.346768


In [22]:
print(f'S 集合中 Outlook 字段的平均信息熵是: {outlook_average_entropy["value"]}')


S 集合中 Outlook 字段的平均信息熵是: 0.6935361388961918


### Step 4: Repeat


## 参考:

- [Using ID3 Algorithm to build a Decision Tree to predict the weather](<https://iq.opengenus.org/id3-algorithm/#:~:text=ID3%20algorithm%2C%20stands%20for%20Iterative,or%20minimum%20Entropy%20(H).>)
- [Wiki-ID3 algorithm](https://en.wikipedia.org/wiki/ID3_algorithm)
